In [28]:
%run config

Code from file 'file:///c%3A/Users/nicholas.radich/Documents/Strava_Lakehouse/config.py':
 client_id = dbutils.secrets.get(scope = "key_vault_secrets", key = "clientid") 
client_secret = dbutils.secrets.get(scope = "key_vault_secrets", key = "clientsecret") 
new_refresh_token = dbutils.secrets.get(scope = "key_vault_secrets", key = "newrefreshtoken")
activity_id_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "activityidpath") 
historical_activity_id_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "historicalactivitydfpath") 
segment_effort_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "segmenteffortpath") 
segment_details_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "segmentdetailspath") 



import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"


payload = {
    'cli

In [27]:
from pyspark.sql.functions import * 
import pandas as pd
import utils

### Logic to orchestrate querying of Strava API

1. Raw query of API to attain all activites currently showing
2. Query DBFS for all activities written to storage
3. Make comparison, and return those activities not wrriten to storage
    - 3a. If all written to storage continue to next comparison
4. For those not written storage, query API and write to storage
    - Repeat 3 to ensure all activites are written

5.  Will now need to compare activities and their associated segments ie each acitvity may have double digit segments

6. Looking at query segment notebook for that comparison - checking all activity IDs have been queried
    - 6a  IF no continue queries, if yes continue to segment details

7. Segment details, will take the longest. Ensure that all segments and their associated details have been queried.    

### Step 1

In [16]:
activity_id_list = [9663381569,9656452945,9635250821, 9578982519,9559341308,9515669005,9408871895
,9298361043,9248492217,9235757648]

activity_id_subset = [9663381569,9656452945,9635250821]

In [20]:
activity_ids_not_written_to_storage = [x for x in activity_id_list if x not in activity_id_subset ]

In [26]:
if len(activity_ids_not_written_to_storage) == 0:
    print ("All activities in storage")
else:
    print( f"Need to query {len(activity_ids_not_written_to_storage)}, activities")

All activities in storage

In [ ]:
# make api call to strava API
#from query_activities notebook
my_dataset = utils.activity_api_call(access_token)

#extract the activities
activity_id_DF, activity_df = extract_activities(my_dataset)

#read in historical activities 
stored_activity_ids = spark.read.format("delta").load(activity_id_path)
activity_id_list = stored_activity_ids.select('activity_id').distinct().rdd.flatMap(lambda x: x).collect()


#make comparison between stored and queries
#need to convert activity_id_DF to a list of IDS
#activity_ids_not_written_to_storage = [x for x in activity_id_list if x not in historical_activity_id_list ]


activity_ids_not_written_to_storage = [x for x in activity_id_subset if x not in activity_id_list ]



#if we do not have all activities written to storage
if len(activity_ids_not_written_to_storage) = 0:
    continue
#take those and query
else: new_activities extract_activities(activity_ids_not_written_to_storage) 
& write_to_storage(new_activities)

done:

Run check again ie if len(activity_ids_not_written_to_storage) = 0:
#Then compare activities with segments to those written in storage

#now looking at 'Query_Segment' notebook 
#will need to compare the activity ids that have already been queried for their segments
segments_in_storage = spark.read.format("delta").load(segment_effort_path)
activity_ids_with_queried_segments = segments_in_storage.select('activity_id').distinct().rdd.flatMap(lambda x: x).collect()


activity_ids_without_queried_segments = [x for x in full_activity_ids  if x not in activity_ids_with_queried_segments ]

#will also need to incorporate activities without segments into the results

#querying historical segments
#will need to make sure that 
segments_in_storage = spark.read.format("delta").load(segment_effort_path)

#make that comparison

#will now need to get the segment details, from 'segment_exploration' notebook
segment_df = query_segment_details(segment_list)

#again making comparison






